In [2]:
from xmljson import badgerfish as bf
from xml.etree.ElementTree import fromstringlist

In [3]:
with open('../wordNet/senses.N.xml') as f:
    data = bf.data(fromstringlist(f.readlines()))

In [4]:
with open('../wordNet/synsets.N.xml') as f:
    syns = bf.data(fromstringlist(f.readlines()))

In [26]:
syns['synsets']['synset']

[OrderedDict([('@id', 'N12658'),
              ('@ruthes_name', 'КОДИРОВАНИЕ ОТ ЗАВИСИМОСТИ'),
              ('@definition', None),
              ('@part_of_speech', 'N'),
              ('sense',
               [OrderedDict([('@id', 115643),
                             ('$', 'МЕДИЦИНСКИЙ КОДИРОВАНИЕ')]),
                OrderedDict([('@id', 115640), ('$', 'КОДИРОВАНИЕ')]),
                OrderedDict([('@id', 115641),
                             ('$', 'КОДИРОВАНИЕ ОТ ЗАВИСИМОСТЬ')]),
                OrderedDict([('@id', 115642),
                             ('$', 'КОДИРОВАНИЕ ЗАВИСИМОСТЬ')])])]),
 OrderedDict([('@id', 'N12659'),
              ('@ruthes_name', 'ПОДГОТОВИТЬ К ИЗВЕСТИЮ'),
              ('@definition',
               'готовить, настраивать соответствующим образом для восприятия чего-либо (какого-либо известия, сообщения и т. п.)'),
              ('@part_of_speech', 'N'),
              ('sense',
               [OrderedDict([('@id', 117307), ('$', 'ПОДГОТОВКА')]),
        

In [38]:
from collections import defaultdict
synsets = defaultdict(list)

for synset in syns['synsets']['synset']:
    if isinstance(synset['sense'], list):
        for sense in synset['sense']:
            synsets[synset['@id']].append(sense['$'].lower())
    else:
        synsets[synset['@id']].append(synset['sense']['$'].lower())

    if synset['@definition'] is not None:
        synsets[synset['@id']].append(synset['@definition'])
        
    if (synset['@ruthes_name'] is not None) and (synset['@ruthes_name'] not in synsets[synset['@id']]):
        synsets[synset['@id']].append(synset['@ruthes_name'].lower())

In [40]:
words = defaultdict(dict)

for sample in data['senses']['sense']:
    if sample['@main_word'] is not None:
        words[sample['@main_word'].lower()][sample['@meaning']] = synsets[sample['@synset_id']]
    else:
        words[sample['@name'].lower()][sample['@meaning']] = synsets[sample['@synset_id']]

In [42]:
words['платформа']

{1: ['грузовой платформа',
  'бортовой платформа',
  'железнодорожный платформа',
  'платформа',
  'платформа для перевозка',
  'грузовая платформа'],
 2: ['грузовой платформа',
  'бортовой платформа',
  'железнодорожный платформа',
  'платформа',
  'платформа для перевозка',
  'грузовая платформа'],
 3: ['платформа',
  'перрон',
  'станционный платформа',
  'посадочный платформа',
  'остановочный платформа',
  'укреплённая платформа, проходящая параллельно железнодорожным путям, предназначенная для посадки и высадки пассажиров поездов',
  'остановочная платформа'],
 4: ['платформа', 'толстая подошва женской обуви', 'платформа (подошва)'],
 5: ['платформа',
  'полустанок',
  'небольшая железнодорожная станция',
  'платформа, полустанок'],
 6: ['платформа общественный организация',
  'программа',
  'программа общественный организация',
  'платформа',
  'программный документ',
  'план действия, деятельности, работы',
  'программа, программный документ'],
 7: ['платформа', 'платформа (воз

In [43]:
sorted(words.items(), key=lambda x: len(x[1]), reverse=True)

[('перевод',
  {1: ['повышение в должность',
    'повышение по служба',
    'повышение работник в должность',
    'перевод с повышение',
    'повышение в должности'],
   2: ['переводческий деятельность',
    'перевод',
    'языковой перевод',
    'переводческий услуга',
    'перевод на другой язык',
    'перевод текст',
    'перевод литература',
    'перевод на другой язык'],
   3: ['сведение',
    'перевод',
    'копировать изображение, обводя, перечерчивая его детали',
    'свести, перевести (копировать изображение)'],
   4: ['переназначение',
    'перевод',
    'назначить на другую должность',
    'переход на другую должность, работу'],
   5: ['переводной текст',
    'перевод',
    'переведенный текст',
    'текст перевод',
    'письменный перевод',
    'результат, продукт перевода\xa0— текст на каком-то языке, соответствующий по смыслу тексту на другом языке и полученный путём его переосмысления и воспроизведения',
    'перевод, текст перевода'],
   6: ['перевод', 'переведение', 'п

# Make embeddings

In [44]:
from gensim.models.wrappers import FastText

In [50]:
model = FastText.load_fasttext_format('../models/fast_text_model.bin')

In [47]:
%load_ext autoreload
%autoreload 2

# Test visualization

In [69]:
vecs_dict = defaultdict(list)
all_vecs = []

for i, w in words['платформа'].items():
    for wi in w:
        vecs_dict[i].append(model[wi])
        all_vecs.append(model[wi])

In [70]:
vecs_dict.keys()

dict_keys([7, 1, 6, 3, 4, 5, 8, 2])

In [71]:
from matplotlib import pyplot as plt
%matplotlib inline

In [72]:
from sklearn.manifold import TSNE

In [73]:
tsne = TSNE(n_components=2)
transformed = tsne.fit_transform(all_vecs)

In [ ]:
for index, words in vecs_dict.items():
    for v in words:
        plt.scatter(v.x, v.y, c)